In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score

import prettytable as pt

In [87]:
df_train = pd.read_csv('covid_train.csv')
df_train.head()

age  sex  cough  fever  chills  sore_throat  headache  fatigue  Urgency
0   30    1      0      0       0            0         0        0        0
1   47    1      0      0       0            0         0        0        0
2   49    1      0      0       0            0         0        0        0
3   50    0      0      0       0            0         0        0        0
4   59    0      0      1       0            0         0        0        0

In [88]:
df_test = pd.read_csv('covid_test.csv')
df_test.head()

age  sex  cough  fever  chills  sore_throat  headache  fatigue  Urgency
0   37    0      1      0       0            0         0        0        0
1   21    1      0      0       0            0         0        0        1
2   20    1      1      1       0            0         0        0        0
3   30    1      0      0       0            0         0        0        1
4   60    0      0      1       0            0         0        0        0

In [89]:
x_train = df_train.drop('Urgency', axis=1)
y_train = df_train['Urgency']

x_test = df_test.drop('Urgency', axis=1)
y_test = df_test['Urgency']

In [90]:
### edTest(test_model) ###

# Define a kNN classification model with k = 7
knn_model = KNeighborsClassifier(n_neighbors=7)

# Fit the above model on the train data
knn_model.fit(x_train, y_train)

# Define a Logistic Regression model with max_iter as 10000 and C as 0.1 (leave all other parameters at default values)
log_model = LogisticRegression(max_iter=10000, C=0.1)

# Fit the Logistic Regression model on the train data
log_model.fit(x_train, y_train)

LogisticRegression(C=0.1, max_iter=10000)

In [91]:
from sklearn.metrics import accuracy_score, confusion_matrix

y_pred_knn = knn_model.predict(x_test)
y_pred_log = log_model.predict(x_test)

def precision_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tp / (tp + fp) if tp + fp > 0 else 0  # Avoid division by zero

def recall_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tp / (tp + fn) if tp + fn > 0 else 0  # Avoid division by zero

def f1_score_func(precision, recall):
    # Using the formula for F1: 2 * (precision * recall) / (precision + recall)
    return 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

knn_precision = precision_score(y_test, y_pred_knn)
log_precision = precision_score(y_test, y_pred_log)

knn_recall = recall_score(y_test, y_pred_knn)
log_recall = recall_score(y_test, y_pred_log)

# Now calculate the F1-score using the custom function
knn_f1 = f1_score_func(knn_precision, knn_recall)
log_f1 = f1_score_func(log_precision, log_recall)

knn_accuracy = accuracy_score(y_test, y_pred_knn)
log_accuracy = accuracy_score(y_test, y_pred_log)

# Specificity calculation
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp) if tn + fp > 0 else 0  # Avoid division by zero

knn_specificity = specificity_score(y_test, y_pred_knn)
log_specificity = specificity_score(y_test, y_pred_log)

metric_scores = {
    "F1-score": [knn_f1, log_f1],
    "Accuracy": [knn_accuracy, log_accuracy],
    "Recall": [knn_recall, log_recall],
    "Specificity": [knn_specificity, log_specificity],
    "Precision": [knn_precision, log_precision]
}

In [92]:
### edTest(test_metrics) ###

table = pt.PrettyTable()
table.field_names = ["Model", "F1-score", "Accuracy", "Recall", "Specificity", "Precision"]
table.add_row(["kNN", round(knn_f1, 3), round(knn_accuracy, 3), round(knn_recall, 3), round(knn_specificity, 3), round(knn_precision, 3)])
table.add_row(["Logistic Regression", round(log_f1, 3), round(log_accuracy, 3), round(log_recall, 3), round(log_specificity, 3), round(log_precision, 3)])
print(table)

+---------------------+----------+----------+--------+-------------+-----------+
|        Model        | F1-score | Accuracy | Recall | Specificity | Precision |
+---------------------+----------+----------+--------+-------------+-----------+
|         kNN         |  0.637   |  0.608   | 0.614  |     0.6     |   0.662   |
| Logistic Regression |  0.648   |  0.592   | 0.671  |    0.491    |   0.627   |
+---------------------+----------+----------+--------+-------------+-----------+
